<a href="https://colab.research.google.com/github/bukhariaujih/KAU-training/blob/master/DME_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount data
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Install depedencies packages

# !pip install scikit-plot
# !pip install pillow
# !pip install scipy
# !pip install scikit-learn
# !apt-get update
# !apt-get upgrade -y
# !apt-get install -y libsm6 libxext6 libxrender-dev
# !pip install opencv-python==3.4.2.17

In [0]:
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Activation
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, confusion_matrix
import imutils 
import scikitplot as skplt

import os
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

import numpy as np

In [0]:
# Load training data
data = './drive/My Drive/training_1/Data/image/data.npy'
target = './drive/My Drive/training_1/Data/image/target.npy'
data = np.load(data)
target = np.load(target)

data = np.squeeze(data, 3)
data = np.repeat(data[..., np.newaxis], 3, -1)


In [0]:
data.shape

In [0]:
def split_validation_set(train, target, test_size):
    random_state = 123
    X_train, X_test, y_train, y_test = train_test_split(train, 
                                                        target, 
                                                        test_size=test_size, 
                                                        random_state=random_state,
                                                        stratify=target)
    print('Split train: ', len(X_train))
    print('Split valid: ', len(X_test))
    return X_train, X_test, y_train, y_test

In [0]:
# Split training data 
X_train, X_test, Y_train, Y_test = split_validation_set(data, 
                                                        target, 
                                                        0.2)

In [0]:
# define model 


In [0]:
model.summary()

In [0]:
# training phase:

history = model.fit(X_train, Y_train, 
                    batch_size=32, 
                    epochs=20, 
                    verbose=1, 
                    validation_data=(X_test, Y_test),
                    shuffle=True)
model.save('/content/drive/My Drive/training_1/Code/image_medical/output/my_model.h5')

In [0]:
# Model performance evaluation:

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [0]:
# Load trained network
model = load_model('/content/drive/My Drive/training_1/Code/image_medical/output/my_model.h5')

In [0]:
# Do prediction on test data
Y_pred=model.predict([X_test])

In [0]:
Y_pred.shape, Y_test.shape

In [0]:
# Performance report:

target_names = ["normal", "dme"]
print(classification_report(np.argmax(Y_test, axis=1), 
                            np.argmax(Y_pred, axis=1), 
                            target_names=target_names))

In [0]:
# Plot roc curve:

skplt.metrics.plot_roc_curve(np.argmax(Y_test, axis=1), Y_pred)
plt.show()

In [0]:
# Show confusion matrix
confusion_matrix(np.argmax(Y_test,axis=1), 
                 np.argmax(Y_pred,axis=1))


In [0]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

# draw the label on the image
for i in range(5):
  # classify the input image
  (normal, dme) = model.predict(X_test)[i]
  label = "NORMAL" if normal > dme else "DME"
  proba = dme if dme > normal else normal
  label = "{}: {:.2f}%".format(label, proba * 100)
  
  image = (X_test[i] * 255).round().astype(np.uint8)
  cv2.putText(image, label, (5, 15),  cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
  
  # show the output image
  cv2_imshow(image)
  

In [0]:
Predicted = np.argmax(Y_pred[:30],axis=1) 
Predicted

In [0]:
GroundTruth = np.argmax(Y_test[:30],axis=1)
GroundTruth

In [0]:
GroundTruth == Predicted